In [ ]:
!wget -O  crowdflower-search-relevance.zip   https://tinyurl.com/y6bg2kqn

In [ ]:
%%bash 
rm -rf crowdflower
mkdir crowdflower 
mv crowdflower-search-relevance.zip crowdflower/
cd crowdflower
unzip crowdflower-search-relevance.zip
unzip test.csv.zip 
unzip train.csv.zip 

In [ ]:
"""
Beating the Benchmark 
Search Results Relevance @ Kaggle
__author__ : Abhishek

"""
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition, pipeline, metrics
from sklearn.model_selection import GridSearchCV


# The following 3 functions have been taken from Ben Hamner's github repository
# https://github.com/benhamner/Metrics
def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)




In [ ]:
train = pd.read_csv('./crowdflower/train.csv')
test = pd.read_csv('./crowdflower/test.csv')
train.head()


In [ ]:
!pip uninstall enum34 -y 
!pip install flair --no-cache-dir
# idx = test.id.values.astype(int)
# train = train.drop('id', axis=1)
# test = test.drop('id', axis=1)

In [ ]:
y = train.median_relevance.values
train = train.drop(['median_relevance', 'relevance_variance'], axis=1)
# test = test.drop(['median_relevance', 'relevance_variance'], axis=1)

In [ ]:
# do some lambda magic on text columns
traindata = list(train.apply(lambda x:'%s %s' % (x['query'],x['product_title']),axis=1))
testdata = list(test.apply(lambda x:'%s %s' % (x['query'],x['product_title']),axis=1))

In [ ]:
traindata

In [ ]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 5), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')
    
    # Fit TFIDF
tfv.fit(traindata)
X =  tfv.transform(traindata) 
X_test = tfv.transform(testdata)

In [ ]:
svd = TruncatedSVD()
    
# Initialize the standard scaler 
scl = StandardScaler()
    
# We will use SVM here..
svm_model = SVC()
    
# Create the pipeline 
clf = pipeline.Pipeline([('svd', svd),
						 ('scl', scl),
                 	     ('svm', svm_model)])
    
    # Create a parameter grid to search for best parameters for everything in the pipeline
param_grid = {'svd__n_components' : [200, 400],
              'svm__C': [10, 12]}

In [ ]:
kappa_scorer = metrics.make_scorer(quadratic_weighted_kappa, greater_is_better = True)
model = GridSearchCV(estimator = clf, param_grid=param_grid, scoring=kappa_scorer,
                                     verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)
model.fit(X, y)
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
	print("\t%s: %r" % (param_name, best_parameters[param_name]))
    
    # Get best model
best_model = model.best_estimator_
    
    # Fit model with best parameters optimized for quadratic_weighted_kappa
best_model.fit(X,y)
preds = best_model.predict(X_test)
    


In [ ]:
master_user="master_user"
master_user_password="master_user_password"
elastic_search_endpoint="elastic_search_endpoint"

In [ ]:
!pip install Elasticsearch

In [ ]:
from __future__ import print_function
from pprint import pprint
import boto3
import json
from elasticsearch import Elasticsearch, RequestsHttpConnection
def connectES(esEndPoint):
    print ('Connecting to the ES Endpoint {0}'.format(esEndPoint))
    try:
        esClient = Elasticsearch(
        hosts=[{'host': esEndPoint, 'port': 443}],
        http_auth=(master_user, master_user_password),
        use_ssl=True,
        verify_certs=True,
        connection_class=RequestsHttpConnection)
        return esClient
    except Exception as E:
        print("Unable to connect to {0}".format(esEndPoint))
        print(E)
        exit(3)

In [ ]:
esClient = connectES(elastic_search_endpoint)

In [ ]:
def indexDocElement(esClient, response):
    try:
        retval = esClient.index(index='crowdflower', doc_type='crowdflower', body=response)
    except Exception as E:
        print("Doc not indexed")
        print("Error: ",E)
        exit(5)

In [ ]:
all_df = pd.concat([train, test])
all_df.head()

In [ ]:
to_index = all_df[["id", "product_title"]]

item_json = to_index.to_json(orient="records")
item_json_arr = json.loads(item_json)
import json 
for i, j in enumerate(item_json_arr):
    indexDocElement(esClient,response=json.dumps(j))

In [ ]:
import requests 
query = "sport earphone"
# query = "woman jeans"
r = requests.get('https://{}/crowdflower/crowdflower/_search?&q=product_title:{}'.format(elastic_search_endpoint,query), auth=(master_user, master_user_password))
rjson = r.json()
rjson 

In [ ]:
import numpy 
to_transform = [] 
for i, r in enumerate(rjson['hits']['hits']): 
    print(r)
    to_transform.append([r['_source']['id'], query+" "+r['_source']['product_title']])

titles = numpy.asarray(to_transform)[:,1]
titles



In [ ]:
title_query_feature =  tfv.transform(titles) 

title_query_feature

In [ ]:
preds = best_model.predict(title_query_feature)

In [ ]:
for i, p in enumerate(preds): 
    to_transform[i].append(p)
    
reranked_result = sorted(to_transform, key=lambda x:x[2], reverse=True)    
    

In [ ]:
reranked_result